In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
import torch, torchvision 
from PIL import Image

In [102]:
from time import sleep
with tqdm(range(10), leave=True) as tq:
    for i in tq:
        j = 0
        sleep(0.01)

In [3]:
import pickle
with open('../saved/datas.pkl', 'rb') as f:
    datas = pickle.load(f)

In [4]:
with open('../saved/header_datas.pkl', 'rb') as f:
    header_datas = pickle.load(f)

In [6]:
import pywt
def cwt(signals,name='', chn=6, wavelet_type = 'morl', vis=False):
    #print(signals.shape)
    # signals: channels x time
    num_steps = signals.shape[1]
    x = np.arange(num_steps) * 0.002
    delta_t = x[1] - x[0] # 500Hz 0.02s
    #print(delta_t)
    #Freq (5, 100)
    if wavelet_type == 'morl':
        scales = np.linspace(8,200,100)
    elif wavelet_type == 'mexh':
        scales = np.linspace(5,50,100)
    elif wavelet_type == 'gaus8':
        scales = np.linspace(12,120,100)
    else: # cmor
        scales = np.linspace(12,120,100)
    

    coef, freqs = pywt.cwt(signals[chn], scales, wavelet_type, delta_t)
    coef2 = (coef-np.min(coef))/(np.max(coef) - np.min(coef))
    #np.save('../saved/cwt/'+name, coef)
    if vis:
        fig = plt.figure()
        ax = fig.add_subplot(211)
        plt.plot(x,signals[chn])

        ax = fig.add_subplot(212)
        #plt.matshow(coefs.astype(float)) 
        plt.imshow(coef.astype(float), aspect='auto', extent=(1, signals.shape[1], freqs[-1], freqs[0]), cmap='jet')
       # plt.colorbar()
        #print((freqs[0], freqs[-1]))
       # plt.yticks(freqs)
        plt.suptitle(name  + ' ' +str(freqs[-1]) + ' '+  str(freqs[0]))
        plt.ylabel('Frequency(Hz)')
        plt.xlabel('Time(ms)')
        plt.show()
        plt.close()
        
    return coef2

In [7]:
from global_vars import labels
def get_basic_info(header_data, labels):
    tmp_hea = header_data[0].split(' ')
    ptID = tmp_hea[0]
    age = None
    sex = None
    label = None
    Dx = None
    for iline in header_data:
        if iline.startswith('#Age'):
            tmp_age = iline.split(': ')[1].strip()
            age = int(tmp_age if tmp_age != 'NaN' else 57)
        elif iline.startswith('#Sex'):
            tmp_sex = iline.split(': ')[1]
            if tmp_sex.strip()=='Female':
                sex =1
            else:
                sex=0
        elif iline.startswith('#Dx'):
            Dx = np.char.rstrip(iline.split(': ')[1].split(','))
            label = [ll in Dx for ll in labels]
            
    
    return [ptID, sex, age, ' '.join(Dx)] + label

In [8]:
from torchvision import datasets, models, transforms
from myeval import binary_acc
import torch.optim as optim
from torch.optim import lr_scheduler
from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import KFold
import time
from myeval import binary_acc_core

st = time.time()
patience = 200
kf = KFold(10)

In [92]:
from torch.utils.data import Dataset
import os
class MyImageDatasetEvaluate(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_imgs):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.infos = infos
        self.data_imgs = data_imgs
        self.transform = torchvision.transforms.ToTensor()

    def __len__(self):
        return len(self.data_imgs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = self.data_imgs[idx]
        sample = self.transform(image)
        return sample
    


In [95]:
cmap = plt.get_cmap('jet')
resize = torchvision.transforms.Resize((244, 244))

infos = []
data_imgs = []
y_pred_res_all = []
labels_all = []
model = load_cwt_nn_model()

for i in tqdm(range(int(6877*0.9), 6877), leave=False):#fDataReNorms.shape[0])): #
    data_imgs, labels = get_image_and_labels(datas[i], header_datas[i])
    image_datasets = MyImageDatasetEvaluate(data_imgs)
    evaluateLoader = torch.utils.data.DataLoader(image_datasets, batch_size=len(data_imgs))
    y_preds = []
    for evaluate_data in evaluateLoader:
        y_preds.append(model(evaluate_data))
    y_pred = y_preds[0]
    y_pred_res = np.max(torch.sigmoid(y_pred).data.numpy(), axis=0)
    y_pred_res_all.append(y_pred_res)
    labels_all.append(labels)

/home/yfeng/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [94]:
len(datas)

6877

In [90]:
model(evaluate_data)

tensor([[-7.7559, -3.8176, -6.3618, -0.7391, -2.8079, -3.8788, -0.6910, -3.4339,
         -2.8801],
        [-6.1261, -3.3991, -5.3951, -0.7411, -2.5121, -3.2138, -1.6194, -1.7641,
         -3.0295],
        [-8.9664, -4.3498, -6.7516, -0.8275, -2.1237, -3.5941, -1.5199, -2.7619,
         -4.2407],
        [-7.9214, -1.2597, -5.8512, -1.4833, -2.0789, -4.3321, -2.9035, -2.9721,
         -4.2069],
        [-6.8842, -3.9893, -6.0467, -1.1785, -3.7997, -2.8877, -2.1973, -1.4071,
         -3.2863]], grad_fn=<AddmmBackward>)

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

def load_cwt_nn_model(model_saved_path='../saved/modelCWTFullBalanced/CWTNetFull0_model.dict'):
    model = models.resnet18(pretrained=True)
#     for param in model.parameters():
#         param.requires_grad = False
    
    num_ftrs = model.fc.in_features
    # change the last output to 9 classes
    model.fc = nn.Linear(num_ftrs, 9)
    
    # load saved model
    model.load_state_dict(torch.load(model_saved_path, map_location=torch.device('cpu')))
    model.eval()
    return model

In [13]:
from global_vars import labels
def get_image_and_labels(data, header_data):
    info = get_basic_info(header_data, labels)
    
    n_length = data.shape[1]
    n_segments = int((n_length)/1000)
    
    coef = cwt(data, str(i)+' '+str(info[3]), chn=0, vis=False)
    data_imgs = []
    for j in range(n_segments):
        s = j*1000
        infos.append(info)
        rgba_img = cmap(coef[:,s:s+1000])
        data_img = Image.fromarray((rgba_img[:, :, :3] * 255).astype(np.uint8))
        data_imgs.append(resize(data_img))
        
    return data_imgs, info[4:]

In [ ]:
def run_12ECG_classifier(data_imgs,model):
    # Use your classifier here to obtain a label and score for each class. 
    
    fDatas = []
    #infos =[]
    fData = filter_data(data[:,1000:], highcut=50.0)
    #fData = datas[idx][:,1000:]
    intervals = sep_rr_interval(fData[:3], height=0.2, distance=200, plot=False)

    # basic info
    #info = get_basic_info(header_datas[idx], labels)
    
    # get data
    #ptID = info[0]
    #print(str(idx) + ' ' + ptID)
    for i in range(len(intervals)):
        l, r = intervals[i]
        fDatas.append(fData[:,l:r])
        #infos.append(info)
            
    fDataRes=np.array([signal.resample(fData, 250, axis=1) for fData in fDatas])
    fDataReNorms=np.array([normalize(fDataRe) for fDataRe in fDataRes])
    
    X = torch.FloatTensor(fDataReNorms)
    
    try:
        output = model(X)
        current_score = np.mean(torch.sigmoid(output).data.numpy(), axis=0)
        current_label = np.rint(current_score).astype(int)
   # print(current_score.shape, current_label.shape)
    except:
        print(fDataRes, header_data)
        current_label = np.zeros((9,), dtype=int)
        current_score = np.zeros((9,))
    return current_label, current_score